In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import missingno as msno

In [ ]:
df = pd.read_csv('/kaggle/input/who-covid19-data-tabe/WHO COVID-19 global table data August 11th 2021 at 10.41.34 AM.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
msno.bar(df,color="dodgerblue", figsize=(15,8), sort="ascending")

*Creating new column for index cause i want to use orignal index column for analysis*

In [ ]:
df.reset_index(inplace=True)
df.rename(columns={'index':'Country','Name':'Region'},inplace=True)

iso_code = pd.read_csv('/kaggle/input/iso-codes/iso_codes.csv') # iso_code dataframe to use for visualization
df = pd.merge(df,iso_code,left_on='Country',right_on='Country')

cordinates = pd.read_csv('/kaggle/input/counties-geographic-coordinates/countries.csv') #latitude & longitude dataframe for visualization
df = pd.merge(df,cordinates,left_on='Country',right_on='name')

df.drop(['country','name'],1,inplace=True)


# Top 10 Countries With Highest Covid Cases In 2021

In [ ]:
most_cases = df.groupby('Country')['Cases - cumulative total'].sum().reset_index().sort_values(by='Cases - cumulative total',ascending=False).head(10)[::-1]
least_cases = df.groupby('Country')['Cases - cumulative total'].sum().reset_index().sort_values(by='Cases - cumulative total',ascending=True).head(10)[::-1]

fig = make_subplots(rows=1, cols=2, column_widths=[0.30, 0.35], vertical_spacing=0.01,
                    subplot_titles=['Top 10 Countries With Highest Covid Cases','Total Covid Cases Boxplot- All Countires'])

fig.append_trace(go.Bar(y=most_cases['Country'],
                       x=most_cases['Cases - cumulative total'],
                       orientation='h',
                       text=most_cases['Cases - cumulative total'],
                       textposition='outside',
                       marker=dict(color='lightpink', line=dict(color='deeppink', width=2)),name=''),1,1)

fig.append_trace(go.Box(y=df['Cases - cumulative total'],
                       marker_color='powderblue',
                       name=''),1,2)


fig.update_layout(
     paper_bgcolor='rgb(248, 248, 255)',
     plot_bgcolor='rgb(248, 248, 255)',
     showlegend=False,
     title_text='Top 10 Countries With Highest Covid Cases',
     title_font_size=22,
width=1000)

In [ ]:
px.set_mapbox_access_token("pk.eyJ1IjoiYXFzYXNhZGFmIiwiYSI6ImNrc241ajRubzA3NjYydmswaXJ1bHlrYTMifQ.WHon85sWay7z-RpuQDKX1w")
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude",     color="Cases - cumulative total", size="Cases - cumulative total",
                  color_continuous_scale=px.colors.cyclical.Twilight, size_max=15, zoom=1,
                       title='Covid Cases Around The World')
fig.update_layout(
     paper_bgcolor='rgb(248, 248, 255)',
     plot_bgcolor='rgb(248, 248, 255)',
     width=1000)
fig.show()

In [ ]:
region_with_most_cases = df.groupby('Region')['Cases - cumulative total'].sum().reset_index().sort_values(by='Cases - cumulative total')


fig = go.Figure()
fig.add_trace(go.Bar(x=region_with_most_cases['Cases - cumulative total'],
                     y=region_with_most_cases['Region'],
                     orientation='h',
                     text=region_with_most_cases['Cases - cumulative total'],
                     marker=dict(
                     color='peachpuff',
                     line=dict(color='black', width=2)
                )))

fig.update_traces(textposition='outside', texttemplate='%{text:.3s}%')

fig.update_layout(title_text='Region With Highest Covid-19 Cases',
                  title_font_size=22,
                  paper_bgcolor='rgb(248, 248, 255)',
                  plot_bgcolor='rgb(248, 248, 255)',
                 width=800)

# Region & Countires With Highest Death Rates Due To Covid-19 In 2021

In [ ]:
most_deaths = df.groupby('Country')['Deaths - cumulative total'].sum().reset_index().sort_values(by='Deaths - cumulative total',ascending=True)[::-1].head(10)

In [ ]:
fig = make_subplots(rows=1, cols=2, column_widths=[0.30, 0.35], vertical_spacing=0.01,
                    subplot_titles=['Top 10 Countries With Highest Rates Of Death Caused By Covid ','Total Death Caused By Covid Cases Boxplot- All Countires'])

fig.append_trace(go.Bar(y=most_deaths['Country'],
                       x=most_deaths['Deaths - cumulative total'],
                       orientation='h',
                       text=most_deaths['Deaths - cumulative total'],
                       textposition='outside',
                       marker=dict(color='plum', line=dict(color='purple', width=2)),name=''),1,1)

fig.append_trace(go.Box(y=df['Deaths - cumulative total'],
                       marker_color='lightgreen',
                       name=''),1,2)


fig.update_layout(
     paper_bgcolor='rgb(248, 248, 255)',
     plot_bgcolor='rgb(248, 248, 255)',
     showlegend=False,
     title_text='Top 10 Countries With Highest Death Rates',
     title_font_size=22,
width=1200)

In [ ]:
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude",
                        color="Deaths - cumulative total", 
                        size="Deaths - cumulative total",
                        color_continuous_scale=px.colors.cyclical.Phase,
                        size_max=15, zoom=1,
                        title = 'Deaths Caused By Covid Around The World')
fig.update_layout(
     paper_bgcolor='rgb(248, 248, 255)',
     plot_bgcolor='rgb(248, 248, 255)',
     width=1000)
fig.show()

In [ ]:
region_with_high_deaths = df.groupby('Region')['Deaths - cumulative total'].sum().reset_index().sort_values(by='Deaths - cumulative total')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=region_with_high_deaths['Deaths - cumulative total'],
                     y=region_with_high_deaths['Region'],
                     orientation='h',
                     text=region_with_high_deaths['Deaths - cumulative total'],
                     marker=dict(
                     color='mintcream',
                     line=dict(color='teal', width=2)
                )))

fig.update_traces(textposition='outside', texttemplate='%{text:.3s}%')

fig.update_layout(title_text='Region With Highest Death Rates Due To Covid-19',
                  title_font_size=22,
                  paper_bgcolor='rgb(248, 248, 255)',
                  plot_bgcolor='rgb(248, 248, 255)',
                  width=700)

# Region & Countries With Highest Newly Reported Cases In Last 7 days

In [ ]:
region_cases_7_days = df.groupby('Region')['Cases - newly reported in last 7 days'].sum().reset_index().sort_values(by='Cases - newly reported in last 7 days')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=region_cases_7_days['Cases - newly reported in last 7 days'],
                     y=region_cases_7_days['Region'],
                     orientation='h',
                     text=region_cases_7_days['Cases - newly reported in last 7 days'],
                     marker=dict(
                     color='azure',
                     line=dict(color='darkblue', width=2)
                )))

fig.update_traces(textposition='outside', texttemplate='%{text:.3s}%')

fig.update_layout(title_text='Region With Highest Newly reported Cases in last 7 days',
                  title_font_size=22,
                  paper_bgcolor='rgb(248, 248, 255)',
                  plot_bgcolor='rgb(248, 248, 255)',
                 width=800)

**Europe region is top on the list with highest newly cases reported in 7 days.**

In [ ]:
most_cases_in_week = df.groupby('Country')['Cases - newly reported in last 7 days'].sum().reset_index().sort_values(by='Cases - newly reported in last 7 days',ascending=True)[::-1].head(10)
least_cases_in_week = df.groupby('Country')['Cases - newly reported in last 7 days'].sum().reset_index().sort_values(by='Cases - newly reported in last 7 days',ascending=False)[::-1].head(10)

In [ ]:
fig = make_subplots(rows=1, cols=2, column_widths=[0.30, 0.35], vertical_spacing=0.01,
                    subplot_titles=['Cases newly reported in last 7 days ','Total Cases newly reported in last 7 days Boxplot- All Countires'])

fig.append_trace(go.Bar(y=most_cases_in_week['Country'],
                       x=most_cases_in_week['Cases - newly reported in last 7 days'],
                       orientation='h',
                       text=most_cases_in_week['Cases - newly reported in last 7 days'],
                       textposition='outside',
                       marker=dict(color='lavenderblush', line=dict(color='pink', width=2)),name=''),1,1)

fig.append_trace(go.Box(y=df['Cases - newly reported in last 7 days'],
                       marker_color='gold',
                       name=''),1,2)


fig.update_layout(
     paper_bgcolor='rgb(248, 248, 255)',
     plot_bgcolor='rgb(248, 248, 255)',
     showlegend=False,
     title_text='Cases - newly reported in last 7 days',
     title_font_size=22,
     width=1200)

In [ ]:
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude",
                        color="Cases - newly reported in last 7 days", 
                        size="Cases - newly reported in last 7 days",
                        color_continuous_scale=px.colors.cyclical.Edge,
                        size_max=15, zoom=1,
                        title = 'Cases - newly reported in last 7 days')
fig.update_layout(
     paper_bgcolor='rgb(248, 248, 255)',
     plot_bgcolor='rgb(248, 248, 255)',
     width=1000)
fig.show()

# Region & Countries With Highest Death Rates Reported In Week

In [ ]:
newly_deaths_7_days = df.groupby('Country')['Deaths - newly reported in last 7 days'].sum().reset_index().sort_values(by='Deaths - newly reported in last 7 days',ascending=False).head(10)[::-1]

fig = make_subplots(rows=1, cols=2, column_widths=[0.30, 0.35], vertical_spacing=0.01,
                    subplot_titles=['Deaths newly reported in last 7 days ','Deaths newly reported in last 7 days Boxplot- All Countires'])

fig.append_trace(go.Bar(y=newly_deaths_7_days['Country'],
                       x=newly_deaths_7_days['Deaths - newly reported in last 7 days'],
                       orientation='h',
                       text=newly_deaths_7_days['Deaths - newly reported in last 7 days'],
                       textposition='outside',
                       marker=dict(color='lightyellow', line=dict(color='yellow', width=2)),name=''),1,1)

fig.append_trace(go.Box(y=df['Deaths - newly reported in last 7 days'],
                       marker_color='silver',
                       name=''),1,2)


fig.update_layout(
     paper_bgcolor='rgb(248, 248, 255)',
     plot_bgcolor='rgb(248, 248, 255)',
     showlegend=False,
     title_text='New Deaths reported in last 7 days',
     title_font_size=22,
width=1200)

In [ ]:
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude",
                        color="Deaths - newly reported in last 7 days", 
                        size="Deaths - newly reported in last 7 days",
                        color_continuous_scale=px.colors.cyclical.Edge,
                        size_max=15, zoom=1,
                        title = 'Deaths - newly reported in last 7 days')
fig.update_layout(
     paper_bgcolor='rgb(248, 248, 255)',
     plot_bgcolor='rgb(248, 248, 255)',
     width=1000)
fig.show()

In [ ]:
region_death_7_days = df.groupby('Region')['Deaths - newly reported in last 7 days'].sum().reset_index().sort_values(by='Deaths - newly reported in last 7 days')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=region_death_7_days['Deaths - newly reported in last 7 days'],
                     y=region_death_7_days['Region'],
                     orientation='h',
                     text=region_death_7_days['Deaths - newly reported in last 7 days'],
                     marker=dict(
                     color='lightpink',
                     line=dict(color='deeppink', width=2)
                )))

fig.update_traces(textposition='outside', texttemplate='%{text:.3s}%')

fig.update_layout(title_text='Region With Highest Deaths newly reported in last 7 days',
                  title_font_size=22,
                  paper_bgcolor='rgb(248, 248, 255)',
                  plot_bgcolor='rgb(248, 248, 255)',
                 width=800)

**Even though Europe region has the most newly reported cases but Americas region is top list with most deaths reported in 7 days.**